<a href="https://colab.research.google.com/github/bitamallisa/Tugas-API/blob/main/bita.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Transformers, Datasets, dan PyTorch (versi GPU jika Colab mendukung GPU)
!pip install transformers datasets torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install optional dependencies untuk evaluasi dan akselerasi
!pip install accelerate evaluate


Looking in indexes: https://download.pytorch.org/whl/cu118
ERROR: Could not find a version that satisfies the requirement datasets (from versions: none)
ERROR: No matching distribution found for datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

In [ ]:
import torch
print(f"GPU Available: {torch.cuda.is_available()}")


GPU Available: True


In [ ]:
from datasets import Dataset
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# 1. Load Dataset
# Menggunakan dataset Hugging Face
dataset = load_dataset("neural-bridge/rag-dataset-12000")

# 2. Preprocessing
# Ambil data
contexts = [str(c) if c is not None else "" for c in dataset['train']['context']]
questions = [str(q) if q is not None else "" for q in dataset['train']['question']]
answers = [str(a) if a is not None else "" for a in dataset['train']['answer']]

# Format input untuk T5
inputs = [f"question: {q} context: {c}" for q, c in zip(questions, contexts)]

# Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Fungsi tokenisasi
def preprocess_data(context, question, answer):
    inputs = tokenizer(f"question: {question} context: {context}",
                       max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(answer, max_length=128, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100  # Mask token padding untuk loss calculation
    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": labels.squeeze()
    }

# Tokenisasi dataset
train_data = [
    preprocess_data(context, question, answer)
    for context, question, answer in zip(contexts[:8000], questions[:8000], answers[:8000])
]
test_data = [
    preprocess_data(context, question, answer)
    for context, question, answer in zip(contexts[8000:], questions[8000:], answers[8000:])
]

# Ubah menjadi Dataset
train_dataset = Dataset.from_dict({
    "input_ids": torch.stack([d["input_ids"] for d in train_data]),
    "attention_mask": torch.stack([d["attention_mask"] for d in train_data]),
    "labels": torch.stack([d["labels"] for d in train_data])
})
test_dataset = Dataset.from_dict({
    "input_ids": torch.stack([d["input_ids"] for d in test_data]),
    "attention_mask": torch.stack([d["attention_mask"] for d in test_data]),
    "labels": torch.stack([d["labels"] for d in test_data])
})

# 3. Load Model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# 4. Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# 6. Train Model
trainer.train()

# 7. Evaluasi Model
results = trainer.evaluate()
print(f"Results: {results}")

# 8. Save Model
model.save_pretrained("./t5_qa_model")
tokenizer.save_pretrained("./t5_qa_model")
print("Model and tokenizer have been saved.")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-d257ab7fa08e>:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.966400,1.169000
2,1.295800,1.139776
3,1.404200,1.133609


Results: {'eval_loss': 1.1336088180541992, 'eval_runtime': 25.1321, 'eval_samples_per_second': 63.664, 'eval_steps_per_second': 7.958, 'epoch': 3.0}
Model and tokenizer have been saved.


In [ ]:
# Load model dan tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "./t5_qa_model"  # Path tempat model tersimpan
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Fungsi untuk menghasilkan jawaban
def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    # Generate answer
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=4,  # Beam search
        early_stopping=True
    )

    # Decode hasilnya
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Contoh penggunaan
context = "Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities. THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years. Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps. Hort Innovation trade manager Jenny Van de Meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017. She said the Australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to $32.6M over the same period. “Australian berry sectors are in a firm position at the moment,” she said. “Production, adoption of protected substrate cropping, improved genetics and an expanding geographic footprint have all helped put Aussie berries on a positive trajectory. “We are seeing a real transition point. Broad industry interest and a strong commercial appetite for export market development combined with the potential to capitalise on existing trade agreements and build new trade partnerships has created this perfect environment for growth.” High-income countries across Europe, North America and Northern Asia have been identified as having a palate for Australian grown berries with more than 4244 tonnes of fresh berries exported in the last financial year alone. The strategy identified the best short-term prospect markets for the Australian blackberry and raspberry industry as Hong Kong, Singapore, The United Arab Emirates and Canada. The strongest short-term trade options identified for the strawberry sector were Thailand, Malaysia, New Zealand and Macau. The strategy focuses heavily on growing the existing strawberry export market from 4 per cent to at least 8 per cent of national production by volume, in markets with a capacity and willingness to pay a premium for quality fruit. For raspberries and blackberries, the sectors aim to achieve a 5 per cent boost in exports assessed by volume across identified markets by 2021. Tasmanian raspberry exporter Nic Hansen said Australia offers some of the sweetest and most attractive berries in the world, and this combined with our stringent food safety standards across all stages of the supply chain puts growers in a solid position. “We have a great product, we are hungry to expand trade and now with this new plan in place, we have a clear roadmap towards driving growth,” Mr Hansen said. He said it is exciting to see new export market prospects for raspberries: “The more options we have for export the better. Now we just have to get on with the job of ensuring industry has all the tools it needs, such as supporting data and relationship building opportunities, to thrive in new markets.” This project was commissioned by Hort Innovation, and developed by market analysts and research consultants Auspex Strategic Advisory and AgInfinity. Hort Innovation will work now with berry sectors to determine levy-funded activities to support trade. See a summary of the strategy on the Hort Innovation website. For more information on the berry industries, refer to the Horticulture Statistics Handbook and the Strategic Investment Plans for strawberries, raspberries and blackberries. Growers seeking more information should email trade@horticulture.com.au"
question = "What is the Berry Export Summary 2028 and what is its purpose??"

answer = generate_answer(question, context)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")


Question: What is the Berry Export Summary 2028 and what is its purpose??
Context: Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities. THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years. Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps. Hort Innovation trade manager Jenny Van de Meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017. She said the Australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to $32.6M over the same period. “Australian berry sectors a

In [ ]:
import torch

model_dir = "./t5_qa_model"
torch.save(model.state_dict(), f"{model_dir}/model_weights.pth")
print("Model weights saved as model_weights.pth")


Model weights saved as model_weights.pth


In [ ]:
from transformers import T5ForConditionalGeneration

model_path = "./t5_qa_model"
model = T5ForConditionalGeneration.from_pretrained("t5-small")  # Muat konfigurasi awal
model.load_state_dict(torch.load(f"{model_path}/model_weights.pth"))
print("Model loaded from model_weights.pth")


<ipython-input-9-ab6efbcbf4db>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_path}/model_weights.pth"))


Model loaded from model_weights.pth


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load tokenizer dan model
model_path = "./t5_qa_model"
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")  # Konfigurasi awal
model.load_state_dict(torch.load(f"{model_path}/model_weights.pth"))
model.eval()  # Set model ke mode evaluasi

# Fungsi untuk menghasilkan jawaban
def generate_answer(question, context):
    # Format input
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    # Generate output
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,  # Panjang maksimum output
        num_beams=4,     # Gunakan beam search
        early_stopping=True
    )

    # Decode output menjadi teks
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Contoh penggunaan
context = "Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities. THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years. Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps. Hort Innovation trade manager Jenny Van de Meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017. She said the Australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to $32.6M over the same period. “Australian berry sectors are in a firm position at the moment,” she said. “Production, adoption of protected substrate cropping, improved genetics and an expanding geographic footprint have all helped put Aussie berries on a positive trajectory. “We are seeing a real transition point. Broad industry interest and a strong commercial appetite for export market development combined with the potential to capitalise on existing trade agreements and build new trade partnerships has created this perfect environment for growth.” High-income countries across Europe, North America and Northern Asia have been identified as having a palate for Australian grown berries with more than 4244 tonnes of fresh berries exported in the last financial year alone. The strategy identified the best short-term prospect markets for the Australian blackberry and raspberry industry as Hong Kong, Singapore, The United Arab Emirates and Canada. The strongest short-term trade options identified for the strawberry sector were Thailand, Malaysia, New Zealand and Macau. The strategy focuses heavily on growing the existing strawberry export market from 4 per cent to at least 8 per cent of national production by volume, in markets with a capacity and willingness to pay a premium for quality fruit. For raspberries and blackberries, the sectors aim to achieve a 5 per cent boost in exports assessed by volume across identified markets by 2021. Tasmanian raspberry exporter Nic Hansen said Australia offers some of the sweetest and most attractive berries in the world, and this combined with our stringent food safety standards across all stages of the supply chain puts growers in a solid position. “We have a great product, we are hungry to expand trade and now with this new plan in place, we have a clear roadmap towards driving growth,” Mr Hansen said. He said it is exciting to see new export market prospects for raspberries: “The more options we have for export the better. Now we just have to get on with the job of ensuring industry has all the tools it needs, such as supporting data and relationship building opportunities, to thrive in new markets.” This project was commissioned by Hort Innovation, and developed by market analysts and research consultants Auspex Strategic Advisory and AgInfinity. Hort Innovation will work now with berry sectors to determine levy-funded activities to support trade. See a summary of the strategy on the Hort Innovation website. For more information on the berry industries, refer to the Horticulture Statistics Handbook and the Strategic Investment Plans for strawberries, raspberries and blackberries. Growers seeking more information should email trade@horticulture.com.au"
question = "What is the Berry Export Summary 2028 and what is its purpose??"

answer = generate_answer(question, context)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")


<ipython-input-11-67d4f14869ad>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_path}/model_weights.pth"))


Question: What is the Berry Export Summary 2028 and what is its purpose??
Context: Caption: Tasmanian berry grower Nic Hansen showing Macau chef Antimo Merone around his property as part of export engagement activities. THE RISE and rise of the Australian strawberry, raspberry and blackberry industries has seen the sectors redouble their international trade focus, with the release of a dedicated export plan to grow their global presence over the next 10 years. Driven by significant grower input, the Berry Export Summary 2028 maps the sectors’ current position, where they want to be, high-opportunity markets and next steps. Hort Innovation trade manager Jenny Van de Meeberg said the value and volume of raspberry and blackberry exports rose by 100 per cent between 2016 and 2017. She said the Australian strawberry industry experienced similar success with an almost 30 per cent rise in export volume and a 26 per cent rise in value to $32.6M over the same period. “Australian berry sectors a